In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [6]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [7]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [8]:
label_map = {0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9}
class_names = ['frog', 'horse', 'ship', 'truck']
cifar10 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0,1,2,3,4,5,6,7,8,9]]
cifar10_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0,1,2,3,4,5,6,7,8,9]]

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [10]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [11]:
import time
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

time1 = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
time2 = time.time()
time3 = time2-time1
print(time3)

Training on device cuda.
2021-04-19 18:28:28.818524 Epoch 1, Training loss 2.0283666259187565
2021-04-19 18:28:45.587640 Epoch 10, Training loss 1.1541462195346424
2021-04-19 18:29:04.678465 Epoch 20, Training loss 0.9853835153152876
2021-04-19 18:29:21.895567 Epoch 30, Training loss 0.9043265613905914
2021-04-19 18:29:38.874878 Epoch 40, Training loss 0.8506361666847678
2021-04-19 18:29:55.848334 Epoch 50, Training loss 0.8108874876861987
2021-04-19 18:30:12.802686 Epoch 60, Training loss 0.7797686700778239
2021-04-19 18:30:30.027643 Epoch 70, Training loss 0.7529088753035001
2021-04-19 18:30:47.533918 Epoch 80, Training loss 0.7283835992255174
2021-04-19 18:31:05.228863 Epoch 90, Training loss 0.7111593760416636
2021-04-19 18:31:21.033840 Epoch 100, Training loss 0.6899750372942757
2021-04-19 18:31:36.732780 Epoch 110, Training loss 0.6753588111885368
2021-04-19 18:31:52.364567 Epoch 120, Training loss 0.659693493448255
2021-04-19 18:32:08.048809 Epoch 130, Training loss 0.6463711944

In [12]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.64


In [16]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [17]:
import time
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

time1 = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
time2 = time.time()
time3 = time2-time1
print(time3)

Training on device cuda.
2021-04-19 18:47:45.419132 Epoch 1, Training loss 2.0419638056279448
2021-04-19 18:48:01.846729 Epoch 10, Training loss 1.199456960572611
2021-04-19 18:48:19.035296 Epoch 20, Training loss 1.0007705829866098
2021-04-19 18:48:36.256423 Epoch 30, Training loss 0.9075732327940519
2021-04-19 18:48:53.689035 Epoch 40, Training loss 0.8528787364892643
2021-04-19 18:49:11.292671 Epoch 50, Training loss 0.8126222433336555
2021-04-19 18:49:28.624747 Epoch 60, Training loss 0.7793253608753005
2021-04-19 18:49:46.010818 Epoch 70, Training loss 0.751642483960637
2021-04-19 18:50:03.337904 Epoch 80, Training loss 0.7293657463453614
2021-04-19 18:50:19.779007 Epoch 90, Training loss 0.7085475045091966
2021-04-19 18:50:35.646689 Epoch 100, Training loss 0.6903051066276668
2021-04-19 18:50:51.871712 Epoch 110, Training loss 0.6735876949546892
2021-04-19 18:51:07.774049 Epoch 120, Training loss 0.6610975086384112
2021-04-19 18:51:23.877664 Epoch 130, Training loss 0.64666188201

In [18]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.79
Accuracy val: 0.63
